# Лабораторная работа №3
## Вариант 4
### 1. Вычислить средние рейтинги для фильма с ID 4 в зависимости от пола

#### 1.1. Считать данные

In [13]:
import pandas as pd

movies = pd.read_table("ml-1m/movies.dat", sep="::",names=["MovieID", "Title", "Genres"], encoding="latin-1")
ratings = pd.read_table("ml-1m/ratings.dat", sep="::", names=["UserID", "MovieID", "Rating", "Timestamp"], encoding="latin-1")
users = pd.read_table("ml-1m/users.dat", sep="::", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"], encoding="latin-1")
display(movies)
display(ratings)
display(users)

/tmp/ipykernel_4298/1859565191.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table("ml-1m/movies.dat", sep="::",names=["MovieID", "Title", "Genres"], encoding="latin-1")
/tmp/ipykernel_4298/1859565191.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table("ml-1m/ratings.dat", sep="::", names=["UserID", "MovieID", "Rating", "Timestamp"], encoding="latin-1")
/tmp/ipykernel_4298/1859565191.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted 

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


#### 1.2. Объединить таблицы

In [14]:
df = ratings.merge(users, on="UserID").merge(movies, on="MovieID")
display(df)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy
1000205,6040,1094,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,956715648,M,25,6,11106,Sophie's Choice (1982),Drama


#### 1.3. Вычислить средний рейтинг по полу

In [15]:
movie_id = 4
result1 = df[df["MovieID"] == movie_id].groupby("Gender")["Rating"].mean()
display(result1)

Gender
F    2.976471
M    2.482353
Name: Rating, dtype: float64

### 2. Отобрать фильмы, получившие не менее 250 оценок, и для них определить средний рейтинг в зависимости от пола

#### 2.1. Сосчитать количество оценок на фильм

In [44]:

rating_counts = (df.groupby("MovieID")).size()
popular_movies = rating_counts[rating_counts >= 250].index

#### 2.2. Отфильтровать популярные фильмы

In [45]:
df_popular = df[df["MovieID"].isin(popular_movies)]

#### 2.3. Сгрупировать по MovieID и полу, посчитать средний рейтинг

In [46]:
gender_ratings = df_popular.groupby(["MovieID", "Gender"])["Rating"].mean().unstack(fill_value=0)

#### 2.4. Привести к нужному формату

In [47]:
result2 = gender_ratings.reset_index()
result2.columns = ["MovieID", "Rating_F", "Rating_M"]

#### 2.5. Добавить Title

In [48]:
result2 = result2.merge(movies[["MovieID", "Title"]], on="MovieID")
result2 = result2[["MovieID", "Title", "Rating_F", "Rating_M"]]
display(result2)

,MovieID,Title,Rating_F,Rating_M
0,1,Toy Story (1995),4.187817,4.130552
1,2,Jumanji (1995),3.278409,3.175238
2,3,Grumpier Old Men (1995),3.073529,2.994152
3,5,Father of the Bride Part II (1995),3.212963,2.888298
4,6,Heat (1995),3.682171,3.909988
...,...,...,...,...
1211,3917,Hellraiser (1987),3.022222,3.128205
1212,3927,Fantastic Voyage (1966),3.535714,3.589041
1213,3948,Meet the Parents (2000),3.619048,3.641838
1214,3949,Requiem for a Dream (2000),3.950000,4.174107


### 3. Найти фильмы, оказавшиеся в топ 5 у зрителей-женщин и худшие 5 у зрителей-мужчин.   Найти   фильмы   (топ   5),   по   которым   мужчины   и   женщины   сильнее   всего разошлись в оценках

#### 3.1. Создать df с фильмами где есть оценки и от M и от F

In [50]:
gender_ratings_full = df_popular.groupby(["MovieID", "Gender"])["Rating"].mean().unstack()
gender_ratings_full = gender_ratings_full.dropna()
display(gender_ratings_full)

Gender,F,M
MovieID,,
1,4.187817,4.130552
2,3.278409,3.175238
3,3.073529,2.994152
5,3.212963,2.888298
6,3.682171,3.909988
...,...,...
3917,3.022222,3.128205
3927,3.535714,3.589041
3948,3.619048,3.641838


#### 3.2. Объединить с movies и добавить заголовок

In [51]:
gender_ratings_full = gender_ratings_full.merge(movies[["MovieID", "Title"]], on="MovieID")
display(gender_ratings_full)

,MovieID,F,M,Title
0,1,4.187817,4.130552,Toy Story (1995)
1,2,3.278409,3.175238,Jumanji (1995)
2,3,3.073529,2.994152,Grumpier Old Men (1995)
3,5,3.212963,2.888298,Father of the Bride Part II (1995)
4,6,3.682171,3.909988,Heat (1995)
...,...,...,...,...
1211,3917,3.022222,3.128205,Hellraiser (1987)
1212,3927,3.535714,3.589041,Fantastic Voyage (1966)
1213,3948,3.619048,3.641838,Meet the Parents (2000)
1214,3949,3.950000,4.174107,Requiem for a Dream (2000)


#### 3.3. Определить топ 5 у женщин

In [52]:
top_female = gender_ratings_full.nlargest(5, "F")[["MovieID", "Title", "F"]]
display(top_female)

,MovieID,Title,F
212,745,"Close Shave, A (1995)",4.644444
336,1148,"Wrong Trousers, The (1993)",4.588235
255,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650
206,720,Wallace & Gromit: The Best of Aardman Animatio...,4.563107
162,527,Schindler's List (1993),4.562602


#### 3.4. Определить 5 худших у мужчин

In [53]:
worst_male = gender_ratings_full.nsmallest(5, "M")[["MovieID", "Title", "M"]]
display(worst_male)

,MovieID,Title,M
1139,3593,Battlefield Earth (2000),1.616949
465,1381,Grease 2 (1982),1.792553
172,546,Super Mario Bros. (1993),1.820339
841,2643,Superman IV: The Quest for Peace (1987),1.847458
502,1556,Speed 2: Cruise Control (1997),1.863014


#### 3.5. Определить топ 5 с наибольшим расхождением между мужчинами и женщинами

In [54]:
gender_ratings_full["Diff"] = gender_ratings_full["F"] - gender_ratings_full["M"]
top_diff = gender_ratings_full.nlargest(5, "Diff")[["MovieID", "Title", "Diff"]]
display(top_diff)

,MovieID,Title,Diff
313,1088,Dirty Dancing (1987),0.830782
796,2468,Jumpin' Jack Flash (1986),0.676359
464,1380,Grease (1978),0.608224
79,261,Little Women (1994),0.548849
1201,3844,Steel Magnolias (1989),0.535777


### 4. Определить топ 5 фильмов с наибольшим разногласием (независимо от пола)

In [62]:
disagreement = df.groupby("MovieID")["Rating"].std().sort_values(ascending=False).head(5)
disagreement = disagreement.to_frame().merge(movies[["MovieID", "Title"]], on="MovieID")
display(disagreement)

,MovieID,Rating,Title
0,572,2.828427,Foreign Student (1994)
1,3800,2.309401,Criminal Lovers (Les Amants Criminels) (1999)
2,1787,2.121320,Paralyzing Fear: The Story of Polio in America...
3,706,2.121320,Sunset Park (1996)
4,3236,2.121320,Zachariah (1971)


### 5. Определить средний рейтинг каждого жанра для различных возрастов. Установите, фильмы какого жанра в каждой категории возрастов имеют наибольший и наименьший рейтинги.

### 5.1. Добавить читаемый возраст

In [63]:
age_map = {
    1: "Under 18",
    18: "18-24",
    25: "25-34",
    35: "35-44",
    45: "45-49",
    50: "50-55",
    56: "56+"
}

df["AgeGroup"] = df["Age"].map(age_map)
display(df)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,AgeGroup
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,Under 18
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,Under 18
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance,Under 18
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama,Under 18
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,Under 18
...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106,Weekend at Bernie's (1989),Comedy,25-34
1000205,6040,1094,5,956704887,M,25,6,11106,"Crying Game, The (1992)",Drama|Romance|War,25-34
1000206,6040,562,5,956704746,M,25,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama,25-34
1000207,6040,1096,4,956715648,M,25,6,11106,Sophie's Choice (1982),Drama,25-34


#### 5.2. Разделить по жанрам

In [64]:
df_exploded = df.copy()
df_exploded["Genres"] = df_exploded["Genres"].str.split("|")
df_exploded = df_exploded.explode("Genres")
display(df_exploded)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,AgeGroup
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,Under 18
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation,Under 18
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Children's,Under 18
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Musical,Under 18
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical,Under 18
...,...,...,...,...,...,...,...,...,...,...,...
1000207,6040,1096,4,956715648,M,25,6,11106,Sophie's Choice (1982),Drama,25-34
1000208,6040,1097,4,956715569,M,25,6,11106,E.T. the Extra-Terrestrial (1982),Children's,25-34
1000208,6040,1097,4,956715569,M,25,6,11106,E.T. the Extra-Terrestrial (1982),Drama,25-34
1000208,6040,1097,4,956715569,M,25,6,11106,E.T. the Extra-Terrestrial (1982),Fantasy,25-34


#### 5.3. Сгрупировать по возрасту и жанру, сосчитать средний рейтинг

In [65]:
genre_age_rating = df_exploded.groupby(["Genres", "AgeGroup"])["Rating"].mean().reset_index()
display(genre_age_rating)

,Genres,AgeGroup,Rating
0,Action,18-24,3.447097
1,Action,25-34,3.453358
2,Action,35-44,3.538107
3,Action,45-49,3.528543
4,Action,50-55,3.611333
...,...,...,...
121,Western,35-44,3.679278
122,Western,45-49,3.667135
123,Western,50-55,3.741322
124,Western,56+,3.792198


#### 5.4. Определить жанр с наибольшим и наименьшим рейтингом по возрастным группам

In [ ]:
extremes_list = []
for age, group in genre_age_rating.groupby("AgeGroup"):
    best = group.loc[group["Rating"].idxmax()]
    worst = group.loc[group["Rating"].idxmin()]
    extremes_list.append({
        "AgeGroup": age,
        "BestGenre": best["Genres"],
        "BestRating": best["Rating"],
        "WorstGenre": worst["Genres"],
        "WorstRating": worst["Rating"]
    })

result3 = pd.DataFrame(extremes_list)
display(result3)

,AgeGroup,BestGenre,BestRating,WorstGenre,WorstRating
0,18-24,Film-Noir,3.997368,Horror,3.172682
1,25-34,Film-Noir,4.058725,Horror,3.200096
2,35-44,Film-Noir,4.064910,Horror,3.276022
3,45-49,Film-Noir,4.105376,Horror,3.262274
4,50-55,Film-Noir,4.175401,Horror,3.158940
5,56+,Film-Noir,4.125932,Horror,3.254401
6,Under 18,Film-Noir,4.145455,Children's,3.241642
